In [ ]:
import asyncio

import panel as pn
from panel.chat import ChatMessage

pn.extension()


The `ChatMessage` is a widget for displaying chat messages with support for various content types.

This widget provides a structured view of chat messages, including features like:
- Displaying user avatars, which can be text, emoji, or images.
- Showing the user's name.
- Displaying the message timestamp in a customizable format.
- Associating reactions with messages and mapping them to icons.
- Rendering various content types including text, images, audio, video, and more.

See [`ChatFeed`](ChatFeed.ipynb) for a structured and straightforward way to build a list of `ChatMessage` objects.

See [`ChatInterface`](ChatInterface.ipynb) for a high-level, *easy to use*, *ChatGPT like* interface.

![Chat Design Specification](../../assets/ChatDesignSpecification.png)

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

##### Core

* **`value`** (object): The message contents. Can be a string, pane, widget, layout, etc.
* **`renderers`** (List[Callable]): A callable or list of callables that accept the value and return a Panel object to render the value. If a list is provided, will attempt to use the first renderer that does not raise an exception. If None, will attempt to infer the renderer from the value.
* **`user`** (str): Name of the user who sent the message.
* **`avatar`** (str | BinaryIO): The avatar to use for the user. Can be a single character text, an emoji, or anything supported by `pn.pane.Image`. If not set, uses the first character of the name.
* **`default_avatars`** (Dict[str, str | BinaryIO]): A default mapping of user names to their corresponding avatars to use when the user is set but the avatar is not. You can modify, but not replace the dictionary. Note, the keys are *only* alphanumeric sensitive, meaning spaces, special characters, and case sensitivity is disregarded, e.g. `"Chat-GPT3.5"`, `"chatgpt 3.5"` and `"Chat GPT 3.5"` all map to the same value.
* **`avatar_lookup`** (Callable): A function that can lookup an `avatar` from a user name. The function signature should be `(user: str) -> Avatar`. If this is set, `default_avatars` is disregarded.
* **`reactions`** (List): Reactions associated with the message.
* **`reaction_icons`** (ChatReactionIcons | dict): A mapping of reactions to their reaction icons; if not provided defaults to `{"favorite": "heart"}`. Provides a visual representation of reactions.
* **`timestamp`** (datetime): Timestamp of the message. Defaults to the instantiation time.
* **`timestamp_format`** (str): The format in which the timestamp should be displayed.

##### Display

* **`show_avatar`** (bool): Whether to display the avatar of the user.
* **`show_user`** (bool): Whether to display the name of the user.
* **`show_timestamp`** (bool): Whether to display the timestamp of the message.
* **`show_reaction_icons`** (bool): Whether to display the reaction icons.
* **`show_copy_icon`** (bool): Whether to show the copy icon.
* **`name`** (str): The title or name of the chat message widget, if any.

___

In [ ]:
ChatMessage(value="Hi and welcome!")

The `ChatMessage` can display any Python object that Panel can display! For example Panel components, dataframes and plot figures.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame({"x": [1, 2, 3], "y": [4, 5, 6]})

fig, axes = plt.subplots()
axes.plot(df["x"], df["y"])
plt.close(fig)

pn.Column(
    ChatMessage(value=pn.widgets.Button(name="Click")),
    ChatMessage(value=df),
    ChatMessage(value=fig),
)

You can specify a custom `user` name and `avatar`

In [ ]:
ChatMessage(value="Want to hear some beat boxing?", user="Beat Boxer", avatar="🎶")

Instead of searching for emojis online, you can also set a personalized emoji avatar by using its name wrapped in `\N{}`!

In [ ]:
ChatMessage(value="Want to hear some beat boxing?", user="Beat Boxer", avatar="\N{musical note}")

You can combine `ChatMessage` with other Panel components as you like.

In [ ]:
pn.Column(
    ChatMessage(
        value="Yes. I want to hear some beat boxing", user="Music Lover", avatar="🎸"
    ),
    ChatMessage(
        value=pn.Column(
            "Here goes. Hope you like this one?",
            pn.pane.Audio(
                "https://upload.wikimedia.org/wikipedia/commons/d/d3/Beatboxset1_pepouni.ogg"
            ),
        ),
        user="Beat Boxer",
        avatar="🎶",
    ),
)

`ChatMessage` can be initialized without any input.

In [ ]:
chat_message = pn.chat.ChatMessage()
chat_message

That way, the `value`, `user`, and/or `avatar` can be dynamically updated either by setting the `value` like this...

In [ ]:
chat_message.value = pn.pane.Markdown("## Cheers!")

Or updating multiple values at once with the `.param.update` method!

In [ ]:
chat_message.param.update(user="Jolly Guy", avatar="🎅")

If you don't specify an `avatar` on construction, then an `avatar` will be looked up in the `default_avatars` dictionary.

In [ ]:
ChatMessage.default_avatars

You can modify the `ChatMessage.default_avatars` in-place.

Note, the keys are *only* alphanumeric sensitive, meaning spaces, special characters, and case sensitivity is disregarded, e.g. `"Chat-GPT3.5"`, `"chatgpt 3.5"` and `"Chat GPT 3.5"` all map to the same value.

In [ ]:
ChatMessage.default_avatars["Wolfram"] = "🐺"
ChatMessage.default_avatars["#1 good-to-go guy"] = "👍"

pn.Column(
    ChatMessage(value="Mathematics!", user="Wolfram"),
    ChatMessage(value="Good to go!", user="#1 Good-to-Go Guy"),
    ChatMessage(value="What's up?", user="Other Guy"),
    max_width=300,
)

The `timestamp` can be formatted using `timestamp_format`.

In [ ]:
pn.chat.ChatMessage(timestamp_format="%b %d, %Y %I:%M %p")

If you'd like a plain interface with only the `value` displayed, set `show_user`, `show_avatar`, and `show_timestamp` to `False` and provide an empty `dict` to `reaction_icons`.

In [ ]:
ChatMessage(
    value="Plain and simple",
    show_avatar=False,
    show_user=False,
    show_timestamp=False,
    reaction_icons={},
)

You can set the usual styling and layout parameters like `sizing_mode`, `height`, `width`, `max_height`, `max_width`, `styles` and `stylesheet`.

In [ ]:
ChatMessage(
    value="Want to hear some beat boxing?",
    user="Beat Boxer",
    avatar="🎶",
    height=250,
    sizing_mode="stretch_width",
    max_width=600,
    styles={"background": "#CBC3E3"},
)

Some active `reactions` can be associated with the message too.

In [ ]:
pn.chat.ChatMessage(value="Love this!", reactions=["favorite"])

If you'd like to display other `reactions_icons`, pass a pair of `reaction` key to tabler `icon` name.

In [ ]:
message = pn.chat.ChatMessage(
    value="Looks good!",
    reactions=["like"],
    reaction_icons={"like": "thumb-up", "dislike": "thumb-down"},
)
message

You may bind a callback to the selected `reactions`.

Here, when the user clicks or sets `reactions`, `print_reactions` activates. 

In [ ]:
def print_reactions(reactions):
    print(f"{reactions} selected.")

pn.bind(print_reactions, message.param.reactions, watch=True)

The easiest and most optimal way to stream output to the `ChatMessage` is through async generators.

If you're unfamiliar with this term, don't fret!

It's simply prefixing your function with `async` and replacing `return` with `yield`.

This example will show you how to **replace** the `ChatMessage` `value`.

In [ ]:
async def replace_response():
    for value in range(0, 28):
        await asyncio.sleep(0.2)
        yield value


ChatMessage(value=replace_response)

This example will show you how to **append** to the `ChatMessage` `value`. 

In [ ]:
sentence = """
    The greatest glory in living lies not in never falling,
    but in rising every time we fall.
"""


async def append_response():
    value = ""
    for token in sentence.split():
        value += f" {token}"
        await asyncio.sleep(0.2)
        yield value


ChatMessage(value=append_response, user="Wise guy", avatar="🤓")


You can also use this to provide parts of responses as soon as they are ready.

In [ ]:
async def respond_when_ready():
    spinner = pn.indicators.LoadingSpinner(value=True, size=25, color="light")
    layout = pn.Column("Love your question. Let me think.", spinner)
    yield layout

    await asyncio.sleep(1)  # Waiting for some external response...
    layout.insert(-1, "I can use Matplotlib to create the Figure")
    yield layout

    await asyncio.sleep(1)  # Calculating ...
    layout.insert(-1, fig)
    layout[-1] = "I hope that answered your question?"
    yield layout


ChatMessage(
    value=respond_when_ready,
    user="Assistant",
    avatar="https://upload.wikimedia.org/wikipedia/commons/6/63/Yumi_UBports.png",
)

For an example on `renderers`, see [ChatInterface](ChatInterface.ipynb).